# GPU Extreme gradient boosting trained on TF-IDF reduced 50 dimensions

1. Same emotion dataset from [NLP-dataset](https://github.com/huseinzol05/NLP-Dataset)
2. Same splitting 80% training, 20% testing, may vary depends on randomness
3. Same regex substitution '[^\"\'A-Za-z0-9 ]+'

## Example

Based on Term-frequency Inverse document frequency

After that we apply SVD to reduce the dimensions, n_components = 50

In [1]:
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
import numpy as np
import re
from sklearn.cross_validation import train_test_split
import sklearn.datasets
from sklearn import pipeline
from sklearn.model_selection import StratifiedKFold

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def clearstring(string):
    string = re.sub('[^\"\'A-Za-z0-9 ]+', '', string)
    string = string.split(' ')
    string = filter(None, string)
    string = [y.strip() for y in string]
    string = ' '.join(string)
    return string

# because of sklean.datasets read a document as a single element
# so we want to split based on new line
def separate_dataset(trainset):
    datastring = []
    datatarget = []
    for i in range(len(trainset.data)):
        data_ = trainset.data[i].split('\n')
        # python3, if python2, just remove list()
        data_ = list(filter(None, data_))
        for n in range(len(data_)):
            data_[n] = clearstring(data_[n])
        datastring += data_
        for n in range(len(data_)):
            datatarget.append(trainset.target[i])
    return datastring, datatarget

In [3]:
trainset_data = sklearn.datasets.load_files(container_path = 'data', encoding = 'UTF-8')
trainset_data.data, trainset_data.target = separate_dataset(trainset_data)

In [4]:
train_X, test_X, train_Y, test_Y = train_test_split(trainset_data.data, trainset_data.target, test_size = 0.2)

In [5]:
decompose = pipeline.Pipeline([('count', TfidfVectorizer()),
                               ('svd', TruncatedSVD(n_components=50))]).fit(trainset_data.data)

In [6]:
params_xgd = {
    'min_child_weight': 10.0,
    'objective': 'multi:softprob',
    'eval_metric': 'mlogloss',
    'num_class': len(trainset_data.target_names),
    'max_depth': 7,
    'max_delta_step': 1.8,
    'colsample_bytree': 0.4,
    'subsample': 0.8,
    'eta': 0.03,
    'gamma': 0.65,
    'num_boost_round' : 700,
    'gpu_id': 0,
    'tree_method': 'gpu_hist'
    }

In [7]:
train_X = decompose.transform(train_X)
test_X = decompose.transform(test_X)

In [8]:
d_train = xgb.DMatrix(train_X, train_Y)
d_valid = xgb.DMatrix(test_X, test_Y)
watchlist = [(d_train, 'train'), (d_valid, 'valid')]
#with open('clf.p', 'rb') as fopen:
#    clf = pickle.load(fopen)
clf = xgb.train(params_xgd, d_train, 100000, watchlist, early_stopping_rounds=100, maximize=False, verbose_eval=50)

[0]	train-mlogloss:1.78223	valid-mlogloss:1.78259
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
[50]	train-mlogloss:1.5574	valid-mlogloss:1.57311
[100]	train-mlogloss:1.49457	valid-mlogloss:1.52355
[150]	train-mlogloss:1.46417	valid-mlogloss:1.50584
[200]	train-mlogloss:1.44302	valid-mlogloss:1.4967
[250]	train-mlogloss:1.42599	valid-mlogloss:1.49095
[300]	train-mlogloss:1.4111	valid-mlogloss:1.48669
[350]	train-mlogloss:1.39748	valid-mlogloss:1.48334
[400]	train-mlogloss:1.38507	valid-mlogloss:1.48065
[450]	train-mlogloss:1.3733	valid-mlogloss:1.47826
[500]	train-mlogloss:1.36218	valid-mlogloss:1.47627
[550]	train-mlogloss:1.3517	valid-mlogloss:1.47462
[600]	train-mlogloss:1.3416	valid-mlogloss:1.47312
[650]	train-mlogloss:1.3319	valid-mlogloss:1.47175
[700]	train-mlogloss:1.32257	valid-mlogloss:1.47055
[750]	train-mlogloss:1.31332	valid-mlogloss:1.46951
[800]	train-mlogloss:1.3

In [9]:
np.mean(test_Y == np.argmax(clf.predict(xgb.DMatrix(test_X), ntree_limit=clf.best_ntree_limit), axis = 1))

0.42349031933015041

In [10]:
from sklearn import metrics
print(metrics.classification_report(test_Y, np.argmax(clf.predict(xgb.DMatrix(test_X), ntree_limit=clf.best_ntree_limit), axis = 1), target_names = trainset_data.target_names))

             precision    recall  f1-score   support

      anger       0.34      0.07      0.11     11336
       fear       0.30      0.06      0.10      9694
        joy       0.46      0.73      0.56     28068
       love       0.17      0.01      0.03      6987
    sadness       0.40      0.54      0.46     24277
   surprise       0.07      0.00      0.01      3000

avg / total       0.37      0.42      0.35     83362



In [12]:
clf.save_model('xgb-tfidf-svd50.model')
bst = xgb.Booster(params_xgd)
bst.load_model('xgb-tfidf-svd50.model')
import json
with open('xgb-tfidf-svd50-param', 'w') as fopen:
    fopen.write(json.dumps(params_xgd))
np.mean(test_Y == np.argmax(bst.predict(xgb.DMatrix(test_X)), axis = 1))

0.42361027806434587